In [ ]:
Kaggle = False
Colab = True

In [ ]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"

    os.chdir(path)
    os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#  if Kaggle:
#     !rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
#     !git clone --recursive https://github.com/Microsoft/LightGBM
#     !apt-get install -y -qq libboost-all-dev
#     %%bash
#     cd LightGBM
#     rm -r build
#     mkdir build
#     cd build
#     cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
#     make -j$(nproc)
#     !cd LightGBM/python-package/;python3 setup.py install --precompile
#     !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
#     !rm -r LightGBM
# else:

# !git clone — recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install — precompile

In [ ]:
# !git clone https://github.com/Microsoft/LightGBM
# %cd /content/drive/My\ Drive/LightGBM/
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [ ]:
%cd /content/drive/My\ Drive/LightGBM/python-package
!sudo python setup.py install

/content/drive/My Drive/LightGBM/python-package
running install
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so', 'compile/lib_lightgbm.so']
running install_egg_info
removing '/usr/local/lib/python3.6/dist-packages/lightgbm-2.3.2-py3.6.egg-info' (and everything under it)
Copying lightgb

In [ ]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Thu Jun 18 23:46:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [ ]:
if Kaggle:
    PATH = '/kaggle/input/'
    outdir = '.'
# PATH = '/Users/helen/Desktop/Data/'
else:
    PATH = 'm5_competition/'
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/lgbm')
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [ ]:
MODELNAME = 'LGBM-GPU'
VERSION = '{}'.format(MODELNAME)
outdir = os.path.join(outdir, VERSION)
if not os.path.exists(outdir):
    os.mkdir(outdir)

from datetime import datetime
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")
# timestampStr = 'all'
outdir = os.path.join(outdir, timestampStr)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "int16", 'snap_TX': 'int16', 'snap_WI': 'int16' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [ ]:

def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int16).min
                      and c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df


In [ ]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv(PATH+ "m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(PATH+"m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    display(cal.head())   
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    # dtype.update({col: "category" for col in catcols})
    dt = pd.read_csv(PATH+"m5-forecasting-accuracy/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()

    # for col in catcols:
    #     dt[col] = dt[col].cat.codes.astype("int16")
    #     dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    # print("step 0")
    # display(dt.head())
    dt = dt.merge(cal, on= "d", copy = False)
    # print("step 1")
    # dt['d'] = dt['d'].astype('category').cat.codes.astype("int16")
    # print(dt['d'].min(), dt['d'].max())
    # print(np.unique(dt['d']))
    # print(len(np.unique(dt['d'])))
    # dt['d'] -= dt['d'].min()
    # print("step 1")
    # display(dt.head())
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    display(dt.head())
    return dt

In [ ]:
def create_fea(dt):
    lags = [7, 28, 91, 182]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28, 91, 182]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [ ]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [ ]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df = reduce_mem_usage(df)
df.shape

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.77


Mem. usage decreased to 2718.23 MB (5.4 % reduction)
CPU times: user 26.8 s, sys: 2.24 s, total: 29.1 s
Wall time: 33.4 s


In [ ]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       int16         
 19  snap_TX       int16         
 20  snap_WI       int16         
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(2), int16(17), object(2)
memory us

In [ ]:
%%time

create_fea(df)
df.shape
df = reduce_mem_usage(df)

Mem. usage decreased to 4504.50 MB (0.0 % reduction)
CPU times: user 9min 32s, sys: 3.15 s, total: 9min 35s
Wall time: 9min 35s


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 45 columns):
 #   Column         Dtype         
---  ------         -----         
 0   id             object        
 1   item_id        int16         
 2   dept_id        int16         
 3   store_id       int16         
 4   cat_id         int16         
 5   state_id       int16         
 6   d              object        
 7   sales          float16       
 8   date           datetime64[ns]
 9   wm_yr_wk       int16         
 10  weekday        int16         
 11  wday           int16         
 12  month          int16         
 13  year           int16         
 14  event_name_1   int16         
 15  event_type_1   int16         
 16  event_name_2   int16         
 17  event_type_2   int16         
 18  snap_CA        int16         
 19  snap_TX        int16         
 20  snap_WI        int16         
 21  sell_price     float16       
 22  lag_7          float16       
 23  lag_2

In [ ]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,lag_91,lag_182,rmean_7_7,rmean_28_7,rmean_91_7,rmean_182_7,rmean_7_28,rmean_28_28,rmean_91_28,rmean_182_28,rmean_7_91,rmean_28_91,rmean_91_91,rmean_182_91,rmean_7_182,rmean_28_182,rmean_91_182,rmean_182_182,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [ ]:
df.dropna(inplace = True)
df.shape

(29681556, 45)

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [ ]:
display(X_train.head())


,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,lag_91,lag_182,rmean_7_7,rmean_28_7,rmean_91_7,rmean_182_7,rmean_7_28,rmean_28_28,rmean_91_28,rmean_182_28,rmean_7_91,rmean_28_91,rmean_91_91,rmean_182_91,rmean_7_182,rmean_28_182,rmean_91_182,rmean_182_182,week,quarter,mday
7111431,1,0,0,0,0,6,1,2013,0,0,0,0,1,0,0,3.970703,0.0,0.0,0.0,1.0,0.142822,0.571289,0.428467,0.285645,0.964355,0.357178,0.250000,0.178589,0.494385,0.285645,0.186768,0.175781,0.335205,0.225220,0.181274,0.170288,2,1,10
7111432,1,0,0,0,0,7,1,2013,0,0,0,0,0,1,1,3.970703,0.0,2.0,0.0,0.0,0.142822,0.856934,0.428467,0.142822,0.964355,0.392822,0.250000,0.178589,0.494385,0.307617,0.186768,0.175781,0.329590,0.236206,0.181274,0.170288,2,1,11
7111438,3,0,0,0,0,6,1,2013,0,0,0,0,1,0,0,4.339844,0.0,1.0,1.0,1.0,0.000000,0.428467,1.571289,0.571289,0.214233,0.321533,1.750000,0.643066,0.593262,0.978027,1.549805,0.912109,1.033203,1.093750,1.230469,1.230469,2,1,10
7111439,3,0,0,0,0,7,1,2013,0,0,0,0,0,1,1,4.339844,0.0,1.0,2.0,0.0,0.000000,0.571289,1.571289,0.571289,0.214233,0.357178,1.786133,0.571289,0.571289,0.978027,1.571289,0.912109,1.033203,1.087891,1.242188,1.219727,2,1,11
7111445,4,0,0,0,0,6,1,2013,0,0,0,0,1,0,0,3.080078,2.0,0.0,2.0,1.0,2.000000,0.714355,0.714355,0.856934,1.463867,1.071289,1.036133,0.964355,1.330078,1.198242,0.846191,0.856934,1.093750,1.010742,0.851562,0.577148,2,1,10


In [ ]:
display(y_train.head())

7111431    0.0
7111432    0.0
7111438    0.0
7111439    0.0
7111445    3.0
Name: sales, dtype: float16

In [ ]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [ ]:
%%time

np.random.seed(777)

# fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
# train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
# train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
#                          categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
#                               categorical_feature=cat_feats,
#                  free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds],free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 14.1 s, sys: 14 ms, total: 14.1 s
Wall time: 14.1 s


In [ ]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 4,
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
    'device': 'gpu'
}

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/content/drive/My Drive/LightGBM/python-package/lightgbm/engine.py:150: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5125
[LightGBM] [Info] Number of data points in the train set: 27681556, number of used features: 39
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 28 dense feature groups (739.18 MB) transferred to GPU in 0.724714 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 0.295419
[20]	valid_0's rmse: 2.878
[40]	valid_0's rmse: 2.56917
[60]	valid_0's rmse: 2.47974
[80]	valid_0's rmse: 2.44732
[100]	valid_0's rmse: 2.43411
[120]	valid_0's rmse: 2.42342
[140]	valid_0's rmse: 2.41553
[160]	valid_0's rmse: 2.40555
[180]	valid_0's rmse: 2.39784
[200]	valid_0's rmse: 2.3893
[220]	valid_0's rmse: 2.38345
[240]	valid_0's rmse: 2.3777
[260]	valid_0's rmse: 2.37261
[280]	valid_0's rmse: 2.36632
[300]	

In [ ]:
m_lgb.save_model(os.path.join(outdir,"model-fd{}.lgb".format(FIRST_DAY)))

In [34]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(os.path.join(outdir, f"submission_{icount}.csv"),index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(os.path.join(outdir,"submission.csv"),index=False)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
1 1.023 0.3333333333333333


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
2 1.018 0.3333333333333333
CPU times: user 6h 58min 5s, sys: 1min 32s, total: 6h 59min 37s
Wall time: 6h 53min


In [35]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,2.103347,1.890740,1.843914,1.648423,2.195484,2.761936,2.357479,2.208293,2.218289,2.335911,2.512687,2.661103,3.747812,3.936665,2.937131,2.550776,2.570686,2.654819,2.975203,4.267691,4.340606,3.489299,3.144475,3.195047,3.182101,3.687521,5.224174,5.595977
1,FOODS_1_001_CA_2_validation,1.883655,2.120204,1.880933,2.818681,2.699841,3.566973,3.698947,2.474488,2.345904,2.388186,2.563968,2.792204,4.446725,4.573107,3.182336,2.954356,3.108206,3.204204,3.531679,5.561607,6.051057,3.818429,3.875778,3.875182,4.035176,4.592534,6.972370,6.860651
2,FOODS_1_001_CA_3_validation,2.036762,1.946710,1.876141,1.877222,1.987243,2.302298,1.649650,1.788696,1.957363,2.132628,2.368447,2.502256,3.279152,3.439992,2.622715,2.438110,2.511840,2.609792,2.790419,3.990891,4.110983,3.045172,2.953196,3.027536,3.075573,3.288632,4.739069,5.445520
3,FOODS_1_001_CA_4_validation,0.774834,0.551585,0.589831,0.609069,0.838398,1.014283,1.079263,1.086141,1.118864,1.031005,1.105758,0.960936,1.051554,1.089644,0.943350,0.885222,0.997111,1.018971,1.077976,1.375213,1.408158,1.048644,0.978943,1.007775,1.036314,1.175038,1.458687,1.599306
4,FOODS_1_001_TX_1_validation,0.330889,0.326637,0.316757,0.326913,0.365867,0.379724,0.464163,1.252654,1.149558,1.076608,1.161126,1.392350,1.468380,1.507492,1.359626,1.405486,1.331446,1.392576,1.367055,1.521929,1.591719,1.152511,1.117417,1.131121,1.203222,1.480199,2.020171,2.257227
5,FOODS_1_001_TX_2_validation,0.832732,0.807404,0.793124,0.783616,0.630614,0.895324,0.778128,0.804044,0.925966,0.883600,1.195201,1.145784,1.288861,1.458846,1.121222,0.998106,1.087252,1.137492,1.261525,1.441567,1.530311,1.203213,1.158286,1.186498,1.199672,1.348079,1.726416,1.832833
6,FOODS_1_001_TX_3_validation,0.858291,0.753475,0.738542,0.887156,0.973739,1.020853,1.056550,0.976187,1.096527,1.011516,1.147338,1.220568,1.433571,1.573672,1.297543,1.096174,1.189357,1.295490,1.378786,1.658875,1.785952,1.369896,1.286893,1.269403,1.337735,1.528469,1.959368,2.094837
7,FOODS_1_001_WI_1_validation,0.528717,0.746332,0.664094,0.702675,0.858881,1.847736,1.509747,1.456749,1.266096,1.367721,1.739181,1.820480,1.794383,1.834772,1.498609,1.295397,1.463762,1.685639,1.920056,2.580983,2.512734,1.813578,1.790241,1.795709,1.896359,2.260693,3.156638,2.976805
8,FOODS_1_001_WI_2_validation,0.700975,0.769963,0.655896,0.626977,0.788152,0.854336,0.793231,0.946940,0.967632,0.874822,1.104120,1.146377,1.252494,1.324032,1.161656,0.996872,1.149042,1.137794,1.169025,1.520113,1.605951,1.148780,1.118109,1.100620,1.084235,1.273856,1.628254,1.674260
9,FOODS_1_001_WI_3_validation,0.468713,0.458347,0.470909,0.491201,0.576298,1.557472,0.948326,1.057630,1.230855,1.097302,1.373610,1.353700,1.199672,1.280206,1.197117,0.945964,1.112616,1.186323,1.255327,1.600530,1.695504,1.215991,1.121638,1.146452,1.165063,1.367293,1.939867,1.878076


In [36]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [37]:
sub.shape

(60980, 29)